In [134]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## Preprocessing

In [77]:
dd =  pd.read_csv("train_clean.csv")

In [78]:
dd

,Unnamed: 0,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
0,41,2000-10-16,0.58820,0.59230,0.57750,0.57850,30.382367,0.602323,0.610376,2.605170,1
1,42,2000-10-23,0.57860,0.58760,0.57310,0.57840,30.343134,0.599133,0.608889,2.598570,1
2,43,2000-10-30,0.57830,0.60440,0.57710,0.59800,45.262621,0.598982,0.608382,2.659958,1
3,44,2000-11-06,0.59820,0.60790,0.59400,0.60400,48.872746,0.599651,0.608179,2.609805,1
4,45,2000-11-13,0.60420,0.60520,0.59410,0.59660,44.936330,0.599244,0.607640,2.586345,1
...,...,...,...,...,...,...,...,...,...,...,...
987,1030,2019-09-30,0.89002,0.89365,0.88333,0.88779,47.151897,0.895761,0.889108,1.564625,0
988,1031,2019-10-07,0.89050,0.90183,0.86835,0.86835,37.551961,0.892106,0.888143,1.760791,0
989,1032,2019-10-14,0.87443,0.88081,0.85808,0.86451,35.993098,0.888426,0.887044,1.830085,0
990,1033,2019-10-21,0.86453,0.86687,0.85770,0.86380,35.698024,0.885143,0.885962,1.776589,0


In [79]:
#Datetime format
dd['Date']= pd.to_datetime(dd['Date'])
dd['Date']=dd['Date'].map(dt.datetime.toordinal)
dd
#How do the outliers affect the volatility? Is it relevant for the network? --- What variables have no outliers?

,Unnamed: 0,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
0,41,730409,0.58820,0.59230,0.57750,0.57850,30.382367,0.602323,0.610376,2.605170,1
1,42,730416,0.57860,0.58760,0.57310,0.57840,30.343134,0.599133,0.608889,2.598570,1
2,43,730423,0.57830,0.60440,0.57710,0.59800,45.262621,0.598982,0.608382,2.659958,1
3,44,730430,0.59820,0.60790,0.59400,0.60400,48.872746,0.599651,0.608179,2.609805,1
4,45,730437,0.60420,0.60520,0.59410,0.59660,44.936330,0.599244,0.607640,2.586345,1
...,...,...,...,...,...,...,...,...,...,...,...
987,1030,737332,0.89002,0.89365,0.88333,0.88779,47.151897,0.895761,0.889108,1.564625,0
988,1031,737339,0.89050,0.90183,0.86835,0.86835,37.551961,0.892106,0.888143,1.760791,0
989,1032,737346,0.87443,0.88081,0.85808,0.86451,35.993098,0.888426,0.887044,1.830085,0
990,1033,737353,0.86453,0.86687,0.85770,0.86380,35.698024,0.885143,0.885962,1.776589,0


In [80]:
#Drop first column of dataframe - MAYBE NOT. MAYBE IT HELPS LEARN THE ORDER? OR IS IT AUTOMATIC ---???
dd = dd.iloc[: , 1:]

In [81]:
##Defining not numeric variables
not_numeric_variables = ['Date']

In [82]:
#Scaling numeric variables
numeric_cols = dd.drop(not_numeric_variables, axis=1).columns

y = dd.iloc[:,dd.shape[1]-1:dd.shape[1]] #Not scale the target variable

dd_scaled = dd.copy()
dd_scaled[numeric_cols]=(dd_scaled[numeric_cols]-dd_scaled[numeric_cols].mean())/dd_scaled[numeric_cols].std()
dd_scaled = dd_scaled.drop(y.columns, axis = 1)
dd_scaled = pd.concat([dd_scaled, y], axis=1)
dd_scaled

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
0,730409,-1.925794,-1.761427,-1.997663,-2.032259,-2.049713,-1.778835,-1.683272,0.319710,1
1,730416,-2.025880,-1.805167,-2.044207,-2.033303,-2.053536,-1.812372,-1.699184,0.315252,1
2,730423,-2.029008,-1.648818,-2.001894,-1.828827,-0.599775,-1.813960,-1.704603,0.356716,1
3,730430,-1.821538,-1.616245,-1.823124,-1.766232,-0.248003,-1.806926,-1.706784,0.322840,1
4,730437,-1.758984,-1.641373,-1.822067,-1.843432,-0.631569,-1.811203,-1.712545,0.306994,1
...,...,...,...,...,...,...,...,...,...,...
987,737332,1.220870,1.043083,1.237436,1.194386,-0.415684,1.306336,1.298820,-0.383131,0
988,737339,1.225875,1.119210,1.078976,0.991580,-1.351105,1.267910,1.288490,-0.250630,0
989,737346,1.058335,0.923588,0.970339,0.951519,-1.503001,1.229225,1.276730,-0.203825,0
990,737353,0.955121,0.793856,0.966319,0.944112,-1.531753,1.194702,1.265164,-0.239959,0


## Models

### Training data set

70% Train, 30% Test

In [83]:
dd_train = dd.sample(frac=0.7,random_state=123)
dd_val = dd.drop(dd_train.index)

In [84]:
dd_train

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
746,735645,0.73990,0.74426,0.73400,0.7395,26.837951,0.762767,0.785421,1.684036,0
532,734147,0.82953,0.84943,0.82856,0.8426,46.154884,0.849721,0.854367,2.221606,1
548,734259,0.88640,0.90407,0.88150,0.8874,60.848157,0.873354,0.862690,1.843366,0
352,732880,0.67300,0.67410,0.66780,0.6735,46.142371,0.676357,0.676583,1.038248,1
684,735211,0.83940,0.84395,0.83500,0.8426,48.905148,0.841863,0.843689,1.377427,0
...,...,...,...,...,...,...,...,...,...,...
954,737101,0.87526,0.88381,0.87448,0.8758,44.445886,0.882478,0.884740,1.533204,0
494,733881,0.88200,0.88400,0.86060,0.8686,41.133004,0.886867,0.885899,2.141572,0
639,734896,0.85350,0.87010,0.85290,0.8690,75.820378,0.826718,0.817782,1.372235,0
464,733671,0.90520,0.92980,0.90020,0.9268,66.375566,0.880589,0.869985,2.210994,0


In [85]:
#Same with scaled data
dd_train_scaled = dd_scaled.sample(frac=0.7,random_state=123) 
dd_val_scaled = dd.drop(dd_train_scaled.index) #should be the same because the random seed is the same than before

In [86]:
dd_train_scaled

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
746,735645,-0.344226,-0.347213,-0.342191,-0.352639,-2.395082,-0.091943,0.189489,-0.302474,0
532,734147,0.590224,0.631551,0.658073,0.722945,-0.512833,0.822279,0.927130,0.060630,1
548,734259,1.183129,1.140057,1.218078,1.190317,0.918885,1.070758,1.016178,-0.194854,0
352,732880,-1.041700,-1.000156,-1.042461,-1.041179,-0.514052,-1.000451,-0.974944,-0.738674,1
684,735211,0.693125,0.580551,0.726196,0.722945,-0.244846,0.739657,0.812889,-0.509574,0
...,...,...,...,...,...,...,...,...,...,...
954,737101,1.066988,0.951508,1.143820,1.069301,-0.679358,1.166688,1.252085,-0.404354,0
494,733881,1.137257,0.953276,0.996996,0.994188,-1.002167,1.212827,1.264486,0.006570,0
639,734896,0.840126,0.823916,0.915544,0.998361,2.377784,0.580426,0.535713,-0.513081,0
464,733671,1.379131,1.379513,1.415888,1.601355,1.457478,1.146821,1.094226,0.053462,0


#### PROBLEM: The sampling doesn't respect the order

In [ ]:
#solve ordering

### Training the models 

#### Model function. 
The following function trains a model (defined by the model_name variable and the choosen parameters) and returns the results in terms of accuracy, sensitivity, sensibility and score (0.6 x sensitivity + 0.4 x specificity).
It uses 10 cross validation.

In [139]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

def model_function_completed(dd, model_name, balancing = "none", sampling ="standard",
                             LDA_priors = None, LDA_shrinkage=None, LDA_solver='svd',
                             QDA_priors = None, QDA_reg_param = -1,
                             SVM_kernel = 'duh', SVM_C = -1, SVM_CW = None, SVM_Gamma = "scale", SVM_Degree=3, SVM_Coef0=0.0, SVM_shrinking = False):
    oversample = RandomOverSampler(sampling_strategy='minority')
    undersample = RandomUnderSampler(sampling_strategy='majority')
    
    accuracy = 0
    sensitivity = 0
    specificity = 0
    score = 0

    #10 CV
    for j in range(10):
        n_cols = dd.shape[1]
        y = dd.iloc[:,n_cols-1:n_cols]
        
        #SAMPLING METHOD
        if (sampling == "standard"):
            train = dd.sample(frac=0.7,random_state=200+j) #random state is a seed value
            test = dd.drop(train.index)
            n_cols = dd.shape[1]
            x_train = train.iloc[:,:n_cols-1]
            y_train = train.iloc[:,n_cols-1:n_cols]
            x_test = test.iloc[:,:n_cols-1]
            y_test = test.iloc[:,n_cols-1:n_cols]
      
        elif (sampling == "balanced_resampling"):
            n_cols = dd.shape[1]
            x_train, x_test, y_train, y_test = train_test_split(dd.iloc[:,:n_cols-1],dd.iloc[:,n_cols-1:n_cols], test_size=0.3, random_state=123+j)

        #for each target
        for i in range(1):

            #BALANCING METHOD
            #oversample
            if (balancing == 'oversample'):
                x_train_balanced, y_train_balanced = oversample.fit_resample(x_train, y_train)
            #undersample
            elif (balancing == 'undersample'):
                x_train_balanced, y_train_balanced = undersample.fit_resample(x_train, y_train)
            #none
            elif (balancing == 'none'):
                x_train_balanced = x_train
                y_train_balanced = y_train

            #MODEL NAME
            if (model_name == "LDA"):
                if (LDA_solver == 'svd'):
                    #shrinkage is not supported for svd solver
                    mod = LinearDiscriminantAnalysis(priors = LDA_priors, solver = LDA_solver)   
                else:
                    mod = LinearDiscriminantAnalysis(priors = LDA_priors, shrinkage = LDA_shrinkage, solver = LDA_solver)
                
            elif (model_name == "QDA"):
                mod = QuadraticDiscriminantAnalysis(priors = QDA_priors, reg_param = QDA_reg_param)
        
            elif (model_name == "SVM"):
                mod = SVC(kernel=SVM_kernel, max_iter = 200, C = SVM_C, class_weight = SVM_CW, gamma = SVM_Gamma, degree = SVM_Degree, coef0 = SVM_Coef0, shrinking = SVM_shrinking)
            
            mod.fit(x_train_balanced, y_train_balanced)
            test_prediction = mod.predict(x_test)
            cm = confusion_matrix(y_test, test_prediction)
        
            sensitivity +=  (cm[1,1]/(cm[1,1]+cm[1,0]))/10
            specificity +=  (cm[0,0]/(cm[0,0]+cm[0,1]))/10
            accuracy += mod.score(x_test, y_test)/10



  
  
    score = 0.6*sensitivity + 0.4*specificity
    """results[i][0] = accuracy[i]
    results[i][1] = sensitivity[i]
    results[i][2] = specificity"""
    
    results = [accuracy,sensitivity,specificity,score]
    return results


#### LDA

##### BASELINE

In [140]:
# LDA BASELINE
from sklearn.metrics import confusion_matrix
#particion test y train, x e y
n_cols = dd.shape[1]
x_train, x_test, y_train, y_test = train_test_split(dd.iloc[:,:n_cols-1],dd.iloc[:,n_cols-1:n_cols], test_size=0.3, random_state=123)

#modelo
mod = LinearDiscriminantAnalysis()
mod.fit(x_train, y_train)
test_prediction = mod.predict(x_test)
cm = confusion_matrix(y_test, test_prediction) 
print(cm)


sensitivity =  (cm[1,1]/(cm[1,1]+cm[1,0]))
specificity =  (cm[0,0]/(cm[0,0]+cm[0,1]))
accuracy = mod.score(x_test, y_test)
print(sensitivity, specificity, accuracy)

[[ 59  85]
 [ 52 102]]
0.6623376623376623 0.4097222222222222 0.540268456375839


##### USING THE FUNCTION

In [141]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [142]:
prints = True
#First hyperparameter: balancing method
for method in ['none']: #undersample has the best performance for LDA  --- , 'oversample', 'undersample']
    if (prints): 
        print("With the following balancing method:", method)
    #Hyperparameter optimitzation:
    #Best score, best hyperparameters, other performance measurements
    max = [0,0,0]
    
    #Hyperparameter values
    solver = ['svd', 'lsqr', 'eigen']
    shrinkage = ['auto']#, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]
    sampling = ["standard"] #"balanced_resampling", 
    
    #Second hyperparameter: solver (solv)
    for solv in solver:
        #Third hyperparameter: shrinkage (s)
        for s in shrinkage:
            #Fourth hyperparameter: priors (p)
            for p in range (1,20): 
                prior =  [0.05*p,1-p*0.05]
                #Fifth hyperparameter: sampling method (sa)
                for sa in sampling:
                    #Training the model with all hyperparameter combinations
                    results = model_function_completed(dd_train, "LDA", LDA_priors = prior, balancing = method, sampling = sa, LDA_shrinkage = s, LDA_solver = solv)
                    #Select the hyperparameters that maximize the performance of the model for each targe
                    if (max[0] < results[3]): #Comparison based in score
                        max[0] = results[3] #Maximum score
                        max[1] = [solv, s, prior[0], sa] #Best solv, s, prior, sa based in score
                        max[2] = [results[0], results[1], results[2]] #Accuracy, sensitivy, specificity
                                  
    print(max)

With the following balancing method: none
[0.6324356313524198, ['lsqr', 'auto', 0.4, 'standard'], [0.5649038461538463, 0.9223293898370044, 0.19759499362554275]]


#### QDA


In [143]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [144]:
prints = True
#First hyperparameter: balancing method
for method in ['none']: #undersample has the best performance for LDA  --- , 'oversample', 'undersample']
    if (prints): 
        print("With the following balancing method:", method)
    #Hyperparameter optimitzation:
    #Best score, best hyperparameters, other performance measurements
    max = [0,0,0]
    
    #Hyperparameter values
    data = ['original', 'scaled']
    sampling = ["balanced_resampling", "standard"] #"balanced_resampling", 
    

    #Second hyperparameter: data (d)
    for d in data:
        #Third hyperparameter: regularization parameter (rp)
        for rp in range (0,20): 
            reg_param = 0.05*rp
            #Fourth hyperparameter: priors (p)
            for p in range (1,20): 
                prior =  [0.05*p,1-p*0.05]
                #Fifth hyperparameter: sampling method (sa)
                for sa in sampling:
                    #Training the model with all hyperparameter combinations
                    if (d == 'original'):
                        results = model_function_completed(dd_train, "QDA", balancing = method, sampling = sa, QDA_priors = prior, QDA_reg_param = reg_param)
                    elif (d == 'scaled'):
                        results = model_function_completed(dd_train_scaled, "QDA", balancing = method, sampling = sa, QDA_priors = prior, QDA_reg_param = reg_param)
                        #Select the hyperparameters that maximize the performance of the model
                        if (max[0] < results[3]): #Comparison based in score
                            max[0] = results[3] #Maximum score
                            max[1] = [d, reg_param, prior[0], sa] #Best solv, s, prior, sa based in score
                            max[2] = [results[0], results[1], results[2]] #Accuracy, sensitivy, specificity
                            
    print(max)

With the following balancing method: none
[0.626477733075052, ['scaled', 0.75, 0.35000000000000003, 'balanced_resampling'], [0.554066985645933, 0.8896062541986948, 0.23178495138958796]]


#### SVM

In [145]:
from sklearn.svm import SVC


###### RBF

In [146]:
kernel = 'sigmoid'
prints = True
#First hyperparameter: balancing method
for method in ['none']:
    if (prints): 
        print("With the following balancing method:", method)
    #Hyperparameter optimitzation:
    #Best score, best hyperparameters, other performance measurements
    max = [0,0,0]
  
    #Hyperparameter values
    gamma =  ['scale', 0.5, 1, 2, 5, 7, 10]
    c_values = [1, 3, 5, 10, 20, 50, 75, 100]
    cw_values = ["balanced", None]
    shrinking = [True, False]
  
    #Second hyperparameter: cost (c)
    for c in c_values: 
        #Third hyperparameter: class weight (cw)
        for cw in cw_values:
            #Fourth hyperparameter: shrinking (s)
            for s in shrinking:
                #Fifth hyperparameter: gamma (g)
                for g in gamma:
                    #Training the model with all hyperparameter combinations
                    results = model_function_completed(dd_train_scaled, "SVM", SVM_kernel=kernel, balancing = method, sampling = "balanced_resampling", SVM_C = c, SVM_CW = cw, SVM_shrinking = s, SVM_Gamma = g)
                    #Select the hyperparameters that maximize the performance of the model
                    if (max[0] < results[3]): #Comparison based in score
                            max[0] = results[3] #Maximum score
                            max[1] = [d, reg_param, prior[0], sa] #Best solv, s, prior, sa based in score
                            max[2] = [results[0], results[1], results[2]] #Accuracy, sensitivy, specificity

    print(max)

With the following balancing method: none
[0.5999999999999999, ['scaled', 0.9500000000000001, 0.9500000000000001, 'standard'], [0.4899521531100478, 0.9999999999999999, 0.0]]


SVM RBF: Has overfitting

###### POLY

In [147]:
kernel = 'poly'
prints = True
#First hyperparameter: balancing method
for method in ['none']:
    if (prints): 
        print("With the following balancing method:", method)
    #Hyperparameter optimitzation:
    #Best score, best hyperparameters, other performance measurements
    max = [0,0,0]
  
    #Hyperparameter values
    gamma =  ['scale', 0.5, 1, 2, 5, 7, 10]
    c_values = [1, 3, 5, 10, 20, 50, 75, 100]
    cw_values = ["balanced", None]
    shrinking = [True, False]
  
    #Second hyperparameter: cost (c)
    for c in c_values: 
        #Third hyperparameter: class weight (cw)
        for cw in cw_values:
            #Fourth hyperparameter: shrinking (s)
            for s in shrinking:
                #Fifth hyperparameter: gamma (g)
                for g in gamma:
                    #Training the model with all hyperparameter combinations
                    results = model_function_completed(dd_train_scaled, "SVM", SVM_kernel=kernel, balancing = method, sampling = "balanced_resampling", SVM_C = c, SVM_CW = cw, SVM_shrinking = s, SVM_Gamma = g)
                    #Select the hyperparameters that maximize the performance of the model
                    if (max[0] < results[3]): #Comparison based in score
                            max[0] = results[3] #Maximum score
                            max[1] = [d, reg_param, prior[0], sa] #Best solv, s, prior, sa based in score
                            max[2] = [results[0], results[1], results[2]] #Accuracy, sensitivy, specificity

    print(max)

With the following balancing method: none
[0.5999999999999999, ['scaled', 0.9500000000000001, 0.9500000000000001, 'standard'], [0.4899521531100478, 0.9999999999999999, 0.0]]


### CONCLUSIONS
Based on the results, we conclude that the LDA, QDA and SVM models do not have sufficient predictive power over the data. The accuracy is slightly over 50%, so it does not offer a significant improvement over a random prediction. With this in mind, we discard these models and do not proceed to evaluate them with the validation data.